In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
!pip install -q scikit-learn
train_dataset=dataset.sample(frac=0.8,random_state=0)
test_dataset=dataset.drop(train_dataset.index)

train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

In [ ]:

num_features = len(train_dataset.keys())

train_dataset_rnn = train_dataset.values.reshape(-1, num_features, 1)
test_dataset_rnn = test_dataset.values.reshape(-1, num_features, 1)


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Identify categorical features
categorical_features = ['sex', 'smoker', 'region']
numerical_features = ['age', 'bmi', 'children'] # Include numerical features

# Create a column transformer for one-hot encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features)],
    remainder='passthrough') # Keep numerical features as they are


In [ ]:
# Apply preprocessing to the training and testing data
train_dataset_processed = preprocessor.fit_transform(train_dataset)
test_dataset_processed = preprocessor.transform(test_dataset) # Use transform on test data

# Convert the processed data back to NumPy arrays
# The ColumnTransformer outputs a sparse matrix if the transformer is OneHotEncoder and remainder='passthrough'
# We need a dense array for the RNN
# The processed data is already a NumPy array in this case, so .toarray() is not needed.
train_dataset_rnn = train_dataset_processed.reshape(-1, train_dataset_processed.shape[1], 1)
test_dataset_rnn = test_dataset_processed.reshape(-1, test_dataset_processed.shape[1], 1)

# Update num_features to reflect the number of features after one-hot encoding
num_features = train_dataset_processed.shape[1]

In [ ]:
model=build_rnn_model((num_features, 1))
model.summary()

In [ ]:
EPOCHS = 1000
history = model.fit(
    train_dataset_rnn, train_labels,
    epochs=EPOCHS, validation_split=0.2, verbose=0,
    callbacks=[tfdocs.modeling.EpochDots()])


In [ ]:
def build_rnn_model(input_shape):
  model = keras.Sequential([
      layers.SimpleRNN(64, activation='relu', input_shape=input_shape),
      layers.Dense(1)
  ])
  optimizer=tf.keras.optimizers.RMSprop(0.001)
  model.compile(loss='mse',
              optimizer=optimizer,
              metrics=['mae', 'mse'])
  return model

In [ ]:
loss, mae, mse = model.evaluate(test_dataset_rnn, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

test_predictions = model.predict(test_dataset_rnn).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)